# Project analysis: human detection pipeline

This Jupyter Notebook serves as a detailed analysis of the results from the **Azure AI human detection pipeline**. Its primary purpose is to load and process the JSON reports generated by the Azure Function. The analysis aims to demonstrate the pipeline's functionality and provide insights into the behaviour of the **Azure AI Vision** service on a sample dataset.